### All Imports

In [ ]:
import numpy as np
import cv2
import os.path
import pickle
import sklearn.model_selection as model_selection

In [ ]:
%reload_ext autoreload
%autoreload 2
import siamese_face as face
import siamese_iris as iris

### Run for Face Mode

In [ ]:
images = pickle.load(
    open(os.path.join('Face_Output', 'cropped_faces.pickle'), 'rb'))
labels = np.load(os.path.join('Face_Output', 'cropped_labels.npy'))

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    images, labels, test_size=0.2
)

model_path = 'face_siamese_model.clf'

### Run for Left Iris Mode

In [ ]:
images = pickle.load(
    open(os.path.join('Iris_Output', 'processed_iries_L.pickles'), 'rb')
)
labels = np.load(os.path.join('Iris_Output', 'processed_labels_L.npy'))

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    images, labels, test_size=0.2
)

model_path = 'iris_siamese_model_L.clf'

### Run for Right Iris Mode

In [ ]:
images = pickle.load(
    open(os.path.join('Iris_Output', 'processed_iries_L.pickles'), 'rb')
)
labels = np.load(os.path.join('Iris_Output', 'processed_labels_R.npy'))

x_train, x_test, y_train, y_test = model_selection.train_test_split(
    images, labels, test_size=0.2
)

model_path = 'iris_siamese_model_R.clf'

### Face Section

In [ ]:
# Create Siamese network branches
base_network = face.create_siamese_network(
    face.input_shape, face.embedding_dim)

input_a = face.layers.Input(shape=face.input_shape)
input_b = face.layers.Input(shape=face.input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = face.tf.abs(processed_a - processed_b)

# Output layer
output = face.layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = face.models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    face.generate_batch(x_train, y_train, face.batch_size),
    steps_per_epoch=len(y_train) // face.batch_size,
    epochs=face.epochs,
    validation_data=face.generate_batch(x_test, y_test, face.batch_size),
    validation_steps=len(y_test) // face.batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    face.generate_batch(x_test, y_test, face.batch_size),
    steps=len(y_test) // face.batch_size
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

### Iris Section

In [ ]:
# Define constants
data_dir = "Dataset/VISA_Iris/VISA_Iris"


# Load and preprocess dataset
class_dirs = os.listdir(data_dir)
pairs = []
labels = []

for class_dir in class_dirs:
    left_images = iris.find_images(os.path.join(data_dir, class_dir, "L"))
    right_images = iris.find_images(os.path.join(data_dir, class_dir, "R"))
    for left_image in left_images:
        for right_image in right_images:
            pairs.append((left_image, right_image))
            labels.append(1 if class_dir == left_image.split("/")[-2] else 0)

# Split dataset into train and test sets
x_train, x_test, y_train, y_test = iris.train_test_split(
    pairs, labels, test_size=0.2, random_state=42)

# Create Siamese network branches
base_network = iris.create_siamese_network(
    iris.input_shape, iris.embedding_dim)
input_a = iris.layers.Input(shape=iris.input_shape)
input_b = iris.layers.Input(shape=iris.input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Calculate L1 distance between the embeddings
distance = iris.tf.abs(processed_a - processed_b)

# Output layer
output = iris.layers.Dense(1, activation='sigmoid')(distance)

# Create Siamese network model
model = iris.models.Model([input_a, input_b], output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])


# Train the model
history = model.fit(
    iris.generate_batch(x_train, y_train, iris.batch_size),
    steps_per_epoch=len(y_train) // iris.batch_size,
    epochs=iris.epochs,
    validation_data=iris.generate_batch(
        x_test, y_test, iris.batch_size),
    validation_steps=len(y_test) // iris.batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    iris.generate_batch(x_test, y_test, iris.batch_size),
    steps=len(y_test) // iris.batch_size
)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")